In [13]:
%matplotlib widget

# Session 2.2

## Multi-material samples with ![gVXR](img/gvxr_logo.png)

## Authors: Ben Thorpe, Iwan Mitchel and Franck Vidal

(version 1.0, 04 Oct 2022)

# Aims of this session

- Create X-ray simulations of samples involving multiple materials
    - Multiple individual materials (e.g. a Copper pipe inside an Aluminium block)
    - Metal Alloys and Mixtures
    - Chemical Compounds
- Visualise the effects of different materials on the resulting X-ray images

## Working With Multiple materials
In the previous notebook we had a single model made of a single material. The plan now is to work up to something a bit more interesting. A multi-part model using multiple different materials. The sample in question is a Turbo pump. This can be found as a 3D model consisting of series of stl files representing the different internal and external parts.

![Screenshot of Paraview](img/Turbo_Pump.png)

## Import packages

To get started we will need to import the required packages. The next cell is the same boilerplate code used in the previous notebook to set everything up for the session. Once again we don't need to know the exact details at this stage so for now we can safely run this cell and move on. However, for the curious the comments explain what the various packages are for.  

In [14]:
import os
import numpy as np # Who does not use Numpy?
import matplotlib # To plot images
import matplotlib.pyplot as plt # Plotting
from matplotlib.colors import LogNorm # Look up table
from matplotlib.colors import PowerNorm # Look up table

font = {'family' : 'serif',
         'size'   : 10
       }
matplotlib.rc('font', **font)

# Uncomment the line below to use LaTeX fonts
# matplotlib.rc('text', usetex=True)

from tifffile import imread, imwrite # Write TIFF files

import base64 # Save the visualisation

from gvxrPython3 import gvxr # Simulate X-ray images
from gvxrPython3.utils import saveProjections # Plot the X-ray image in linear, log and power law scales
from gvxrPython3.utils import compareWithGroundTruth # Plot the ground truth, the test image, and the relative error map in %
from gvxrPython3.utils import interactPlotPowerLaw # Plot the X-ray image using a Power law look-up table
from gvxrPython3.utils import visualise # Visualise the 3D environment if k3D is supported
#from gvxrPython3.utils import plotScreenshot # Visualise the 3D environment if Matplotlib is supported

## Create an OpenGL context

As with the previous notebook we will need to create an OpenGL context using `gvxr.createWindow(window_id, visible, backend, opengl_major_version, opengl_minor_version)`

For this tutorial (and the subsequent ones) we will stick with **"EGL"** for the backend, this is because we are using the Cloud for this training. On my PC and laptop, I would use "OPENGL" to enable an interactive 3D visualisation window. 

In [15]:
print("Create an OpenGL context")

window_id = 0
opengl_major_version = 4
opengl_minor_version = 5

# gvxr.createOpenGLContext(window_id, opengl_major_version, opengl_minor_version);

# backend = "OPENGL"
# visible = True

# gvxr.createWindow(window_id, visible, backend, opengl_major_version, opengl_minor_version);

visible = False
# gvxr.createWindow(window_id, visible, backend, opengl_major_version, opengl_minor_version);

backend = "EGL"
# visible has no effect with EGL
gvxr.createWindow(window_id, visible, backend, opengl_major_version, opengl_minor_version);

Create an OpenGL context


Fri Oct  7 15:28:14 2022 ---- Create window (ID: 0)
Fri Oct  7 15:28:14 2022 (WW) Window 0 already exists.


## Setting up the Scene
Now that we have our OpenGL context we need to import the different pieces of the model and set the parameters for the source, detector ect.
---
## Task

Using knowledge from, the previous notebook setup the scene as follows:
- place a 300 KeV monochromatic X-Ray source of 100 photons at `x = 0`, `y = -40.0 cm`, `z = 0.0 cm`
- place the X-Detector at `x = 0`, `y = -40.0 cm`, `z = 0.0 cm`
- set the Detector to have 800 pixels in x and y spaced 0.7 mm apart in both directions. Also set the upVector to be along the negative z-axis i.e. (0.0,0.0,-1.0)
- Import the 6 stl files, found in `input_data/TurboPump` each with appropriate unique id.
- place the centre of the bounding box for each part at `x = 0`, `y = 20.0 cm`, `z = 0.0 cm`

In [16]:
gvxr.setSourcePosition(0.0 ,-40.0,0.0,"cm")
gvxr.usePointSource()
gvxr.setMonoChromatic(0.3,"MeV",1000)
gvxr.setDetectorNumberOfPixels(800,800)
gvxr.setDetectorPixelSize(0.7,0.7,"mm")
gvxr.setDetectorPosition(0.0,30.0,0.0,"cm")
gvxr.setDetectorUpVector(0,0,-1)

parts_list = ["internals","coupler","front_flange","rear_flange","housing","roller_bearing"]

part_files = ["input_data/TurboPump/internals.stl",
               "input_data/TurboPump/coupler.stl",
               "input_data/TurboPump/front_flange.stl",
               "input_data/TurboPump/rear_flange.stl",
               "input_data/TurboPump/housing.stl",
               "input_data/TurboPump/ThrustRollerBearing.stl"]
               
for i,name in enumerate(parts_list):
     gvxr.loadMeshFile(name,part_files[i],"mm")
     gvxr.translateNode(name,0.0,-20.0,0.0,"cm")
     gvxr.moveToCentre()
     gvxr.displayScene()
     
#plotScreenshot()


Fri Oct  7 15:28:15 2022 ---- Initialise the renderer
Fri Oct  7 15:28:15 2022 ---- file_name:	input_data/TurboPump/internals.stl	nb_faces:	87620	nb_vertices:	262860	bounding_box (in cm):	(-3.48613, -5.55, -3.48613)	(3.48613, -0.735, 3.48613)
Fri Oct  7 15:28:15 2022 (WW) Rendering in an hidden window may work, it may not.
Fri Oct  7 15:28:15 2022 ---- file_name:	input_data/TurboPump/coupler.stl	nb_faces:	2532	nb_vertices:	7596	bounding_box (in cm):	(-4.13387, -11.365, -4.13387)	(4.13387, -5.55, 4.13387)
Fri Oct  7 15:28:15 2022 (WW) Rendering in an hidden window may work, it may not.
Fri Oct  7 15:28:15 2022 ---- file_name:	input_data/TurboPump/front_flange.stl	nb_faces:	6120	nb_vertices:	18360	bounding_box (in cm):	(-11.425, 0, -11.425)	(11.425, 3.535, 11.425)
Fri Oct  7 15:28:15 2022 (WW) Rendering in an hidden window may work, it may not.
Fri Oct  7 15:28:15 2022 ---- file_name:	input_data/TurboPump/rear_flange.stl	nb_faces:	10516	nb_vertices:	31548	bounding_box (in cm):	(-11.425, 

## Working with basic Chemical Elements

We know from the theory that the attenuation of the X-Ray beam is directly dependant on:
- The beam energy
- The material density
- The materials Z number.

Thus by changing the material of the sample we will change the amount of attenuation, and hence end up with a lighter, or darker image.

![Periodic table](img/03-PeriodicTable.png)[^1]

[^1]: Image of periodic table from https://sciencenotes.org/wp-content/uploads/2014/11/PeriodicTableWallpaper1.png

---
## Task
We saw in the previous notebook how to use the `setElement(ID,Element)` function to change the element that the sample is made from. Remember `ID` is a string that acts a label for the model. whilst `element` can be the symbol, full name or atomic number of any of the first 100 elements in the periodic table.

For this task we have deliberately chosen an beam energy such that light elements will be appear quite transparent. Whilst heavier elements will appear opaque.

With this in mind try setting different part of the model to different elements and computing the resulting xray images. We suggest setting the internal structures (internals.stl, coupler.stl and ThrustRollerBearing.stl) to something nice and heavy like Iron. whilst the external parts can be set to something lighter like Titanium, Aluminum or even Carbon to allow you to see the parts inside. 

Also as a bonus exercise try imaging the model from, that is rotated 90 degrees around the x-axis.

In [19]:
materials = ["Fe","Fe","Al","Al","Al","Fe"]

for i,name in enumerate(parts_list):
     gvxr.setElement(name, materials[i])

#plotScreenshot()
xray_image_front = np.array(gvxr.computeXRayImage()).astype(np.single)

# for i,name in enumerate(parts_list):
#      gvxr.moveToCentre()
#      gvxr.translateNode(name,0.0,0.0,4.0,"cm")
#      gvxr.rotateNode(name, 90, 1, 0, 0)

# xray_image_side = np.array(gvxr.computeXRayImage()).astype(np.single)

plt.figure(figsize=(10, 5))
plt.title("Image simulated using gVirtualXray\nusing a linear colour scale")
plt.imshow(xray_image_front, cmap="gray")
plt.colorbar(orientation='vertical');
plt.margins(0,0)

RuntimeError: Error: No properties set
	- in File: /io/gvxr/src/Utilities/PhotonCrossSection.cxx
	- in Function: getDensity
	- at Line: 228


## Alloys and Mixtures

Note: According to [World Materail](https://www.theworldmaterial.com/stainless-steel-chemical-composition/) US SAE standard 305 stainless steel consists of 0.12% Carbon, 2.00% Manganese, 0.03% Sulphur, 1.00% Si, 19.0% Chromium, 11.0% Ni with the rest Iron (66.85%)

In [ ]:
gvxr.terminate()